# Voici un notebook de démarrage pour vous aider à débuter le hackathon.

**[Inspiré par l'article de blog d'AI Geek (wishesh)](https://levelup.gitconnected.com/building-a-private-ai-chatbot-2c071f6715ad)**

In [ ]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [1]:
# Installer les packages requis
%%capture
!pip install -q datasets transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers gradio
!pip install -U langchain-community -q
!pip install pdfplumber -q
!pip install PyMuPDF -q
!pip install pypdf -q

In [2]:
!pip install -U langchain-huggingface -q

In [3]:
# importer les libraries requises
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.llms import HuggingFacePipeline
from huggingface_hub import notebook_login
from langchain import HuggingFacePipeline
from langchain.vectorstores import Chroma
from datasets import load_dataset
import gradio as gr
import chromadb
import locale
import torch
import os
import json

In [4]:
# Pour corriger une erreur Unicode dans Google Colab
locale.getpreferredencoding = lambda: "UTF-8"

In [5]:
# Assurez-vous de créer un compte Hugging Face et de générer votre jeton pour vous connecter ici.
notebook_login()

# Charger le modèle

In [8]:
# Spécifiez le nom du modèle Huggingface
model_name = "meta-llama/Llama-2-7b-chat-hf"
#model_name = "mistralai/Mistral-7B-Instruct-v0.1"

###### other models:
# "Trelis/Llama-2-7b-chat-hf-sharded-bf16"
# "bn22/Mistral-7B-Instruct-v0.1-sharded"
# "mistralai/Mistral-7B-Instruct-v0.2"
# "HuggingFaceH4/zephyr-7b-beta"
# "anakin87/zephyr-7b-alpha-sharded"
# function for loading 4-bit quantized model

def load_quantized_model(model_name: str):
    """
    :param model_name: Nom ou chemin du modèle à charger.
    :return: Modèle quantifié chargé.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        # Charger en 4 bits
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

# Fonction pour initialiser le tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialise le tokenizer avec le nom de modèle spécifié.

    :param model_name: Nom ou chemin du modèle pour l'initialisation du tokenizer.
    :return: Tokenizer initialisé.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Définir l'id du token de début de phrase
    return tokenizer


In [9]:
# Charger le modèle
model = load_quantized_model(model_name)

# Initialiser le tokenizer
tokenizer = initialize_tokenizer(model_name)

# Spécifier les ids des tokens de fin
stop_token_ids = [0]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Créer des embeddings

###### Charger les données

In [10]:
# En supposant des données localement
# Décompresser le dossier des données
!unzip -q /content/Data.zip

In [11]:
DATA_PATH = "/content/Data"
JSONS_PATH = f"{DATA_PATH}/data_json_indabax_cameroon"
PDFS_PATH = f"{DATA_PATH}/data_pdfs_indabax_cameroon"

In [12]:
pdfs = [doc for doc in os.listdir(PDFS_PATH) if doc.endswith('.pdf')]
print(f"Il y a {len(pdfs)} fichiers pdf dans le {PDFS_PATH}.")

Il y a 11 fichiers pdf dans le /content/Data/data_pdfs_indabax_cameroon.


In [13]:
jsons = [doc for doc in os.listdir(JSONS_PATH) if doc.endswith('.json')]
print(f"Il y a {len(jsons)} fichiers json dans le répertoire {JSONS_PATH}.")

Il y a 1184 fichiers json dans le répertoire /content/Data/data_json_indabax_cameroon.


In [14]:
# Code montrant comment extraire du texte d'un fichier pdf en utilisant pdfplumber.
import pdfplumber

example_pdf = f"{PDFS_PATH}/{pdfs[0]}"
print(f"Traitement en cours : {example_pdf}")

pdf_text = ""
try:
    with pdfplumber.open(example_pdf) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:  # Si du texte est extrait, l'ajouter à pdf_text
                pdf_text += text + "\n"
except Exception as e:
    print(f"Erreur lors de l'ouverture du fichier PDF : {e}")

# Affichage des 1000 premiers caractères du texte extrait
print(pdf_text[:1000])


Traitement en cours : /content/Data/data_pdfs_indabax_cameroon/AU_arbitrage(1).pdf
www.Droit-Afrique.com OHADA
OHADA
Acte uniforme sur le droit de l’arbitrage
Acte adopté le 11 mars 1999 et paru au JO OHADA n°08 du 15 mai 1999
Index
[NB - les chiffres renvoient aux numéros des articles]
 Application de l’Acte uniforme : 34 et 35  Instance arbitrale :
 Arbitrabilité d’un litige : 1, 2 - acquiescement : 16
 Arbitre : - délibérations : 18, 19
- amiable composition : 15 - désistement : 16
- décès : 8 - fin : 16
- démission : 8 - liaison : 10
- dessaisissement : 22 - mise en délibéré : 17
- droits civils : 6 - nouvelle : 29
- durée de la mission : 12  Juge étatique :
- incapacité : 8 - assistance judiciaire : 14, 23
- mission : 26 - désignation des arbitres : 8
- nomination : 5, 7 - exécution provisoire : 28
- personne physique : 6 - exequatur : 30
- pouvoirs : 14, 15 - incompétence pour trancher le litige : 13
- récusation : 7, 8 - mesures provisoires : 13
- remplacement : 5 - po

In [15]:
import fitz  # PyMuPDF

# Charger le fichier PDF
doc = fitz.open(f"{PDFS_PATH}/{pdfs[0]}")

# Extraire les métadonnées
metadata = doc.metadata
print("📄 Métadonnées du PDF :")
for key, value in metadata.items():
    print(f"{key}: {value}")

# Extraire le texte complet de toutes les pages
full_text = ""
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    full_text += f"\n--- Page {page_num + 1} ---\n{text}"

# Imprimer ou sauvegarder le texte extrait
print("\n📝 Texte extrait :")
print(full_text)

# Optionnel : Fermer le document
doc.close()

📄 Métadonnées du PDF :
format: PDF 1.5
title: OHADA - Acte uniforme du 11 mars 1999 sur le droit de l'arbitrage (www.droit-afrique.com)
author: Droit-Afrique
subject: 
keywords: 
creator: PScript5.dll Version 5.2.2
producer: Acrobat Distiller 10.0.0 (Windows)
creationDate: D:20120411085446+02'00'
modDate: D:20150907093910+02'00'
trapped: 
encryption: None

📝 Texte extrait :

--- Page 1 ---
 
 
www.Droit-Afrique.com 
 
OHADA 
Acte Uniforme sur le droit de l’arbitrage 
1/6
 
OHADA 
 
Acte uniforme sur le droit de l’arbitrage 
 
Acte adopté le 11 mars 1999 et paru au JO OHADA n°08 du 15 mai 1999 
 
 
 
Index  
 
 
[NB - les chiffres renvoient aux numéros des articles] 
 
 

Application de l’Acte uniforme : 34 et 35 

Arbitrabilité d’un litige : 1, 2 

Arbitre : 
- 
amiable composition : 15 
- 
décès : 8 
- 
démission : 8 
- 
dessaisissement : 22 
- 
droits civils : 6 
- 
durée de la mission : 12 
- 
incapacité : 8 
- 
mission : 26 
- 
nomination : 5, 7 
- 
personne physique : 6 
- 


In [16]:
import json

# Exemple de code montrant comment extraire le contenu d'un fichier json.
def extract_json_content(json_path):
    # Lire et traiter le fichier json
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    return json_data

# Exemple de fichier json à traiter
example_json = f"{JSONS_PATH}/{jsons[0]}"
print(f"Traitement en cours : {example_json}")

# Extraction du contenu du fichier json
json_content = extract_json_content(example_json)

# Afficher le contenu extrait du json
print(json_content)

Traitement en cours : /content/Data/data_json_indabax_cameroon/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20220609-1022022.json
{'title': "| OHADA, Cour commune de justice et d'arbitrage, 09 juin 2022, 102/2022", 'content': "ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICE(CCJA)Deuxième chambreAudience publique du 09 juin 2022Pourvoi : n° 225/2021/PC du 15/06/2021Affaire : Monsieur Aa A(Conseil : Me Souleyemane COULIBALY, Avocat à la Cour)ContreSociété Mamadou DIARRA SA(Conseils : Cabinet d’avocats SOW &amp; Associés, Avocats à la Cour)Arrêt N° 102/2022 du 09 juin 2022La Cour Commune de Justice et d’ Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxième Chambre, présidée par Monsieur Armand Claude DEMBA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 09 juin 2022, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Messieurs Ar

### Traitement des fichiers JSON

La fonction ci-dessous montre comment traiter et charger les fichiers JSON dans un objet Dataset adapté à la construction de la base de données vectorielle.

In [17]:
import os
import json
from datasets import Dataset

# Fonction pour traiter le contenu du fichier JSON
def process_json_data(json_data):
    # Adaptez ceci en fonction de la structure de votre fichier JSON
    # En supposant que le JSON contient des clés comme 'title', 'content' et 'details'
    title = json_data.get('title', 'Aucun Titre')  # Par défaut, 'Aucun Titre' si non trouvé
    content = json_data.get('content', 'Aucun Contenu')  # Par défaut, 'Aucun Contenu' si non trouvé
    details = json_data.get('details', 'Aucun Détail')  # Par défaut, 'Aucun Détail' si non trouvé

    return {'title': title, 'content': content, 'details': details}

# Répertoire contenant vos fichiers JSON
json_directory = JSONS_PATH

# Liste pour contenir les données traitées
jsons_data = []

# Itérer sur tous les fichiers JSON dans le répertoire
for json_file in os.listdir(json_directory):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_directory, json_file)

        # Lire et traiter chaque fichier JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

        # Traiter les données JSON et les ajouter à la liste
        processed_data = process_json_data(json_data)
        jsons_data.append(processed_data)

# Optionnel: Créer un objet Dataset avec les données traitées
dataset = Dataset.from_dict({'data': jsons_data})

# Afficher les premières lignes du Dataset pour vérifier
print(dataset[:5])

{'data': [{'content': "ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICE(CCJA)Deuxième chambreAudience publique du 09 juin 2022Pourvoi : n° 225/2021/PC du 15/06/2021Affaire : Monsieur Aa A(Conseil : Me Souleyemane COULIBALY, Avocat à la Cour)ContreSociété Mamadou DIARRA SA(Conseils : Cabinet d’avocats SOW &amp; Associés, Avocats à la Cour)Arrêt N° 102/2022 du 09 juin 2022La Cour Commune de Justice et d’ Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxième Chambre, présidée par Monsieur Armand Claude DEMBA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 09 juin 2022, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Messieurs Armand Claude DEMBA,       PrésidentBirika jean Claude BONZI,      JugeCésar Apollinaire ONDO MVE,   JugeArsène Jean Bruno MINIME,     Juge, rapporteurSabiou MAMANE NAISSA,     JugeSur le renvoi enreg

In [18]:
# Maintenant, 'data' contient une liste de dictionnaires comme
# {'title': 'Titre 1', 'content': 'Contenu 1', 'details': 'Détails 1'}
print(jsons_data[:2])  # Afficher les deux premiers éléments traités

[{'title': "| OHADA, Cour commune de justice et d'arbitrage, 09 juin 2022, 102/2022", 'content': "ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICE(CCJA)Deuxième chambreAudience publique du 09 juin 2022Pourvoi : n° 225/2021/PC du 15/06/2021Affaire : Monsieur Aa A(Conseil : Me Souleyemane COULIBALY, Avocat à la Cour)ContreSociété Mamadou DIARRA SA(Conseils : Cabinet d’avocats SOW &amp; Associés, Avocats à la Cour)Arrêt N° 102/2022 du 09 juin 2022La Cour Commune de Justice et d’ Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxième Chambre, présidée par Monsieur Armand Claude DEMBA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 09 juin 2022, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Messieurs Armand Claude DEMBA,       PrésidentBirika jean Claude BONZI,      JugeCésar Apollinaire ONDO MVE,   JugeArsène Jean Bruno MI

In [19]:
# Vérifier le contenu d'un élément de données donné
jsons_data[0]['title']

"| OHADA, Cour commune de justice et d'arbitrage, 09 juin 2022, 102/2022"

In [20]:
jsons_data[0]['content']

"ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICE(CCJA)Deuxième chambreAudience publique du 09 juin 2022Pourvoi : n° 225/2021/PC du 15/06/2021Affaire : Monsieur Aa A(Conseil : Me Souleyemane COULIBALY, Avocat à la Cour)ContreSociété Mamadou DIARRA SA(Conseils : Cabinet d’avocats SOW &amp; Associés, Avocats à la Cour)Arrêt N° 102/2022 du 09 juin 2022La Cour Commune de Justice et d’ Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxième Chambre, présidée par Monsieur Armand Claude DEMBA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 09 juin 2022, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Messieurs Armand Claude DEMBA,       PrésidentBirika jean Claude BONZI,      JugeCésar Apollinaire ONDO MVE,   JugeArsène Jean Bruno MINIME,     Juge, rapporteurSabiou MAMANE NAISSA,     JugeSur le renvoi enregistré au greffe de la 

In [21]:
jsons_data[0]['details']

"Références :\nOrigine de la décision\nPays :\nOHADA\nJuridiction :\nJuridiction :\nCour commune de justice et d'arbitrage\nDate de la décision :\n09/06/2022\nDate de l'import :\n17/04/2023\nNumérotation\nNuméro d'arrêt : 102/2022\nIdentifiant URN:LEX : urn:lex;ohada;cour.commune.justice.arbitrage;arret;2022-06-09;102.2022"

## Chargement des fichiers JSON dans un Dataset

Les données ont deux formats : pdfs et jsons. Le code ci-dessous montre comment charger les fichiers json pour create un object Dataset.

In [22]:
from datasets import Dataset, DatasetDict

# Créer un Dataset à partir de la liste de dictionnaires
data_set = Dataset.from_dict({
    'title': [entry['title'] for entry in jsons_data],
    'content': [entry['content'] for entry in jsons_data],
    'details': [entry['details'] for entry in jsons_data],
})

# Diviser le dataset en train (ou autres splits si nécessaire)
data_set = DatasetDict({
    'train': data_set,
})

# Maintenant, le dataset_dict devrait contenir la structure souhaitée
print(data_set)

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'details'],
        num_rows: 1184
    })
})


In [23]:
data_set['train'][0].keys()

dict_keys(['title', 'content', 'details'])

In [24]:
print(data_set['train'][0]['title'])

| OHADA, Cour commune de justice et d'arbitrage, 09 juin 2022, 102/2022


In [25]:
print(data_set['train'][0]['content'])

ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICE(CCJA)Deuxième chambreAudience publique du 09 juin 2022Pourvoi : n° 225/2021/PC du 15/06/2021Affaire : Monsieur Aa A(Conseil : Me Souleyemane COULIBALY, Avocat à la Cour)ContreSociété Mamadou DIARRA SA(Conseils : Cabinet d’avocats SOW &amp; Associés, Avocats à la Cour)Arrêt N° 102/2022 du 09 juin 2022La Cour Commune de Justice et d’ Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxième Chambre, présidée par Monsieur Armand Claude DEMBA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 09 juin 2022, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Messieurs Armand Claude DEMBA,       PrésidentBirika jean Claude BONZI,      JugeCésar Apollinaire ONDO MVE,   JugeArsène Jean Bruno MINIME,     Juge, rapporteurSabiou MAMANE NAISSA,     JugeSur le renvoi enregistré au greffe de la C

In [26]:
print(data_set['train'][0]['details'])

Références :
Origine de la décision
Pays :
OHADA
Juridiction :
Juridiction :
Cour commune de justice et d'arbitrage
Date de la décision :
09/06/2022
Date de l'import :
17/04/2023
Numérotation
Numéro d'arrêt : 102/2022
Identifiant URN:LEX : urn:lex;ohada;cour.commune.justice.arbitrage;arret;2022-06-09;102.2022


In [27]:
# texts = []  # Initialize texts as an empty list to store file contents

# for number in range(len(data_set['train'])):
#   content = data_set['train'][number]['content']
#   texts.append(content)

# print(f"The json folder consists of {len(texts)} texts.")

In [28]:
#print(texts[0])

In [29]:
#print(texts[-1])

## Extraire les textes des fichiers PDF

___Tâche___ : Lire et traiter les fichiers PDF du dossier pdf de manière similaire.


In [30]:
# import os
# import pdfplumber

# # Fonction pour extraire le texte des fichiers PDF
# def extraire_texte_pdf(pdf_path):
#     """
#     Ouvre un fichier PDF et extrait le texte de toutes ses pages.
#     :param pdf_path: Chemin du fichier PDF à traiter
#     :return: Texte extrait de toutes les pages du PDF
#     """
#     texte_complet = ""
#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             texte = page.extract_text()
#             if texte:
#                 texte_complet += texte + "\n"
#     return texte_complet

# # Dossier contenant les fichiers PDF
# pdf_directory = PDFS_PATH

# # Liste pour stocker les textes extraits des fichiers PDF
# pdf_texts = []

# # Itérer sur tous les fichiers PDF dans le dossier
# for pdf_file in os.listdir(pdf_directory):
#     if pdf_file.endswith(".pdf"):
#         pdf_path = os.path.join(pdf_directory, pdf_file)

#         # Lire et extraire le texte de chaque fichier PDF
#         texte_pdf = extraire_texte_pdf(pdf_path)

#         # Ajouter le texte extrait à la liste
#         pdf_texts.append({
#             'filename': pdf_file,
#             'text': texte_pdf
#         })

# # Afficher les premiers textes extraits
# print(pdf_texts[:2])  # Afficher les deux premiers éléments extraits


In [31]:
## extend the texts list with the pdf_texts list
# texts.extend(pdf_texts)
# print(f"The length of texts is now {len(texts)}.")

In [32]:
# Charger tous les fichiers PDF dans le répertoire
loader = PyPDFDirectoryLoader(PDFS_PATH)

# Cela renverra une liste d'objets Document
pdf_documents = loader.load()

In [33]:
# Afficher un document d'exemple
pdf_documents[0]

Document(metadata={'producer': 'Acrobat Distiller 10.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2012-04-11T08:54:46+02:00', 'author': 'Droit-Afrique', 'moddate': '2015-09-07T09:39:10+02:00', 'title': "OHADA - Acte uniforme du 11 mars 1999 sur le droit de l'arbitrage (www.droit-afrique.com)", 'source': '/content/Data/data_pdfs_indabax_cameroon/AU_arbitrage(1).pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='www.Droit-Afrique.com  OHADA \nActe Uniforme sur le droit de l’arbitrage 1/6\n \nOHADA \n \nActe uniforme sur le droit de l’arbitrage \n \nActe adopté le 11 mars 1999 et paru au JO OHADA n°08 du 15 mai 1999 \n \n \n \nIndex  \n \n \n[NB - les chiffres renvoient aux numéros des articles] \n \n \n\uf0b7\uf020 Application de l’Acte uniforme : 34 et 35 \n\uf0b7\uf020 Arbitrabilité d’un litige : 1, 2 \n\uf0b7\uf020 Arbitre : \n- amiable composition : 15 \n- décès : 8 \n- démission : 8 \n- dessaisissement : 22 \n- droits civils : 6 \n- dur

In [34]:
# Contenu d'un pdf
print(pdf_documents[0].page_content)

www.Droit-Afrique.com  OHADA 
Acte Uniforme sur le droit de l’arbitrage 1/6
 
OHADA 
 
Acte uniforme sur le droit de l’arbitrage 
 
Acte adopté le 11 mars 1999 et paru au JO OHADA n°08 du 15 mai 1999 
 
 
 
Index  
 
 
[NB - les chiffres renvoient aux numéros des articles] 
 
 
 Application de l’Acte uniforme : 34 et 35 
 Arbitrabilité d’un litige : 1, 2 
 Arbitre : 
- amiable composition : 15 
- décès : 8 
- démission : 8 
- dessaisissement : 22 
- droits civils : 6 
- durée de la mission : 12 
- incapacité : 8 
- mission : 26 
- nomination : 5, 7 
- personne physique : 6 
- pouvoirs : 14, 15 
- récusation : 7, 8 
- remplacement : 5 
- révocation : 5, 8 
- saisine : 10 
 Commerce international : 15 
 Contradictoire (principe du) : 14, 26 
 Convention d’arbitrage : 
- forme écrite : 3, 26 
- interprétation : 4 
- indépendance du contrat principal : 4 
- nullité : 4, 13, 26 
- preuve : 3 
- moment de la souscription : 4 
- validité : 4, 11 
 Conventions internationales : 3

In [35]:
def extract_from_document(document):
    # Extraire le titre du document. Utiliser le titre dans les métadonnées ou la première ligne du contenu de la page
    title = document.metadata.get("title") or document.page_content.split("\n")[0]

    # Extraire les détails du document à partir des métadonnées
    details = {
        "author": document.metadata.get("author"),  # Auteur du document
        "creation_date": document.metadata.get("creationdate"),  # Date de création
        "producer": document.metadata.get("producer"),  # Producteur du document
        "creator": document.metadata.get("creator"),  # Créateur du document
        "modification_date": document.metadata.get("moddate"),  # Date de modification
        "source": document.metadata.get("source"),  # Source du document
    }

    # Nettoyer et extraire le contenu de la page
    content = document.page_content.strip()

    # Retourner un dictionnaire avec le titre, les détails et le contenu extraits
    return {
        "title": title,
        "details": details,
        "content": content
    }

In [36]:
doc = extract_from_document(pdf_documents[0])
doc['title']

"OHADA - Acte uniforme du 11 mars 1999 sur le droit de l'arbitrage (www.droit-afrique.com)"

In [37]:
print(doc['content'])

www.Droit-Afrique.com  OHADA 
Acte Uniforme sur le droit de l’arbitrage 1/6
 
OHADA 
 
Acte uniforme sur le droit de l’arbitrage 
 
Acte adopté le 11 mars 1999 et paru au JO OHADA n°08 du 15 mai 1999 
 
 
 
Index  
 
 
[NB - les chiffres renvoient aux numéros des articles] 
 
 
 Application de l’Acte uniforme : 34 et 35 
 Arbitrabilité d’un litige : 1, 2 
 Arbitre : 
- amiable composition : 15 
- décès : 8 
- démission : 8 
- dessaisissement : 22 
- droits civils : 6 
- durée de la mission : 12 
- incapacité : 8 
- mission : 26 
- nomination : 5, 7 
- personne physique : 6 
- pouvoirs : 14, 15 
- récusation : 7, 8 
- remplacement : 5 
- révocation : 5, 8 
- saisine : 10 
 Commerce international : 15 
 Contradictoire (principe du) : 14, 26 
 Convention d’arbitrage : 
- forme écrite : 3, 26 
- interprétation : 4 
- indépendance du contrat principal : 4 
- nullité : 4, 13, 26 
- preuve : 3 
- moment de la souscription : 4 
- validité : 4, 11 
 Conventions internationales : 3

In [38]:
print(doc['details'])

{'author': 'Droit-Afrique', 'creation_date': '2012-04-11T08:54:46+02:00', 'producer': 'Acrobat Distiller 10.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'modification_date': '2015-09-07T09:39:10+02:00', 'source': '/content/Data/data_pdfs_indabax_cameroon/AU_arbitrage(1).pdf'}


In [39]:
print(doc['details']['source'])

/content/Data/data_pdfs_indabax_cameroon/AU_arbitrage(1).pdf


## Construire une classe Document pour une base de données clé-vecteur

In [40]:
class Document:
    def __init__(self, content, metadata=None):
        """
        Initialiser un document avec son contenu et ses métadonnées.

        :param content: Contenu du document (texte ou autre format)
        :param metadata: Métadonnées associées au document (facultatif, un dictionnaire vide si non fourni)
        """
        # Initialiser le contenu de la page du document
        self.page_content = content

        # Initialiser les métadonnées comme un dictionnaire vide si aucun n'est fourni
        self.metadata = metadata if metadata else {}

    def update_metadata(self, key, value):
        """
        Mettre à jour les métadonnées du document.

        :param key: La clé des métadonnées à ajouter ou modifier
        :param value: La valeur associée à la clé
        """
        self.metadata[key] = value

    def get_metadata(self):
        """
        Récupérer les métadonnées du document.

        :return: Dictionnaire des métadonnées
        """
        return self.metadata

    def get_content(self):
        """
        Récupérer le contenu du document.

        :return: Contenu du document
        """
        return self.page_content

    def __str__(self):
        """
        Retourner une représentation sous forme de chaîne de caractères du document.

        :return: Une chaîne représentant le titre et un extrait du contenu
        """
        return f"Document: {self.metadata.get('title', 'Sans titre')} - Extrait: {self.page_content[:100]}..."


In [41]:
# Construire les documents a partir the contenus json.

json_documents = [
    Document(content=data['content'], metadata={"title": data['title'], "details": data['details']})
    for data in data_set['train']
]

In [42]:
# Examinons un document
print(json_documents[0])

Document: | OHADA, Cour commune de justice et d'arbitrage, 09 juin 2022, 102/2022 - Extrait: ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICE(CCJA...


In [43]:
# json_documents = [Document(content=text) for text in texts]

# Combiner les documents JSON et les documents PDF
documents = json_documents + pdf_documents

# Diviser les documents en petits morceaux
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)  # Modifiez le chunk_size et le chunk_overlap selon les besoins
all_splits = text_splitter.split_documents(documents)


In [44]:
# spécifier le modèle d'encodage (en utilisant le transformateur de phrase de huggingface)

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)


<ipython-input-44-aead8a2ba6e4>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [45]:
# Exemple de texte pour obtenir l'embedding
text_to_embed = "Ceci est un exemple de texte pour obtenir un embedding."

# Obtenez l'embedding pour le texte
embedding = embeddings.embed_documents([text_to_embed])

# Affichez l'embedding
print(embedding)

[[-0.031500693410634995, -0.049962181597948074, 0.004357101861387491, 0.04359627515077591, -0.028997870162129402, 0.00850200280547142, 0.039947837591171265, 0.07866194099187851, 0.005423740483820438, -0.0007118612993508577, 0.039831679314374924, 0.02603030949831009, 0.012480353936553001, 0.0367625430226326, 0.029276421293616295, -0.05994526296854019, -0.017683643847703934, 0.018972592428326607, -0.011209827847778797, 0.012791404500603676, 0.022388296201825142, 0.013701763935387135, 0.01805119588971138, 0.015774231404066086, 0.026193635538220406, 0.03534393757581711, -0.0063852667808532715, -0.05070710927248001, 0.00973291601985693, -0.022381853312253952, -0.0014772039139643312, 0.01547527126967907, 0.05369507893919945, -0.05834585055708885, 1.6655454828651273e-06, 0.01438409835100174, -0.03820003196597099, 0.0064317574724555016, -0.006832440849393606, -0.04389461502432823, 0.013295079581439495, -0.024139169603586197, -0.0028641093522310257, -0.0194380059838295, -0.002373382216319442, 0

In [46]:
folder_path2 = './Vector_db/'  # changez cela par le chemin dans votre disque où vous souhaitez stocker la base de données vectorielle

# intégrer les morceaux de documents
# les embeddings sont stockés dans une base de données (probablement sur disque) spécifiée par persist_directory
# assurez-vous que folder_path2 est correctement défini et pointe vers un répertoire valide où vous avez des permissions d'écriture
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory= folder_path2 + "chroma_db")
retrieverts = vectordb.as_retriever()

# Comment charger
# vectordbts = Chroma(persist_directory= folder_path2 + "chroma_db", embedding_function=embeddings)


# Créer un pipeline

In [51]:
def extract_answer(raw_text: str) -> str:
    try:
        marker = "\nHelpful Answer:"
        if marker in raw_text:
            return raw_text.split(marker)[1].strip()
        else:
            return raw_text.strip()
    except Exception as e:
        return f"Erreur d'extraction: {str(e)}"

In [77]:
# Création d'un pipeline de génération de texte
text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# spécifier le modèle LLM
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

def create_conversation(query: str, chat_history: list, language: str = "French") -> tuple:
    try:
        # Objet pour stocker l'historique des conversations
        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )

        # Configuration de la chaîne de question-réponse
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retrieverts,
            memory=memory,
            get_chat_history=lambda h: h,
        )

        # Construction d’un prompt simple et naturel
        prompt = (
            f"Question : {query}\n"
            f"Réponds en {language}, de façon concise et précise."
        )

        # Exécution (on n’envoie plus l’historique explicitement : la mémoire s’en charge)
        result = qa_chain({"question": prompt})

        # Extraction de la réponse utile et mise à jour de l'historique
        answer = extract_answer(result["answer"])
        chat_history.append((query, answer))
        return answer, chat_history

    except KeyError as ke:
        err = f"Clé manquante : {ke}"
    except ConnectionError as ce:
        err = f"Erreur de connexion : {ce}"
    except Exception as e:
        err = f"Erreur inattendue : {e}"

    # En cas d’erreur, on l’ajoute à l’historique et on renvoie une chaîne vide
    chat_history.append((query, err))
    return "", chat_history



Device set to use cuda:0


In [78]:
test_query = "Qu'est-ce que le CCJA ?"
test_history = []
create_conversation(test_query, test_history)

<ipython-input-77-d795133dbc8d>:30: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": prompt})


("Le CCJA est la Cour Commune de Justice et d'Arbitrage de l'Organisation pour l'Harmonisation en Afrique du Droit des Affaires (OHADA). Elle est compétente pour traiter les litiges relatifs à l'application des textes unificats de l'OHADA, notamment les contrats de transport routier.",
 [("Qu'est-ce que le CCJA ?",
   "Le CCJA est la Cour Commune de Justice et d'Arbitrage de l'Organisation pour l'Harmonisation en Afrique du Droit des Affaires (OHADA). Elle est compétente pour traiter les litiges relatifs à l'application des textes unificats de l'OHADA, notamment les contrats de transport routier.")])

In [65]:
test_history

[("Qu'est-ce que le CCJA ?",
  "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nLa CCJA est incompétente pour un litige relatif au paiement d’une prime exceptionnelle et de salaires réclamés par un travailleur à son ancien employeur, devant les juridictions sociales, lesquelles ont statué en application du seul droit national ; les dispositions de l’AUSCGIE invoquées par la demanderesse aussi bien devant le juge d’instance que la cour d’appel, ayant été déclarées inapplicables dans le litige par les juges du fond, après appréciation souveraine des faits qui leur étaient soumis.ARTICLE 14 TRAITE OHADACCJA, Ass. plén., n° 138/2015 du 19 novembre 2015 ; P. n° 034/2010/PC du 23/03/2010 : BIAO-Côte d’Ivoire dite BIAO-CI c/ Allou TOGNAN Ernest.Arrêt N° 138/2015 du 19 novembre 2015La Cour Commune de Justice et d’Arbitrage (CCJA) de l’Organisation pourl’Harmonisation en Afrique

In [55]:
from IPython.display import HTML

HTML("<span style='color:red; font-weight:bold'>L'extrait de code commenté dans la cellule suivante n'est pas censé être correct. Il vous donne simplement une idée de la manière de générer votre fichier de soumission.</span>")


In [ ]:
# import pandas as pd

# ## Constructing the submission csv
# submission_rows = []

# qid = "Q1"
# submission_rows.append({"ID": f"{qid}_Answer", "Target": answer})
# submission_rows.append({"ID": f"{qid}_Reference_Document", "Target": reference_document})
# submission_rows.append({"ID": f"{qid}_Article_Number", "Target": article_number})

# # turn submission_rows in a pd
# submission_df = pd.DataFrame(submission_rows)
# submission_df.to_csv("submission.csv", index=False)

# submission_df

In [56]:
import pandas as pd

def create_submission_file(questions_and_answers):
    """
    Creates a submission CSV file from a list of questions and answers.

    Args:
        questions_and_answers (list): A list of dictionaries, where each dictionary
            contains the question ID ('qid'), answer, reference document, and
            article number.

    Returns:
        pd.DataFrame: The submission DataFrame.
    """
    submission_rows = []
    for qa_item in questions_and_answers:
        qid = qa_item['qid']
        submission_rows.append({"ID": f"{qid}_Answer", "Target": qa_item['answer']})
        submission_rows.append({"ID": f"{qid}_Reference_Document", "Target": qa_item['reference_document']})
        submission_rows.append({"ID": f"{qid}_Article_Number", "Target": qa_item['article_number']})

    submission_df = pd.DataFrame(submission_rows)
    submission_df.to_csv("submission.csv", index=False)
    return submission_df


In [57]:
import pandas as pd
test_df = pd.read_csv("Test.csv")
test_df.head()

,ID,Question
0,Q1,Quels types d'entités sont soumises aux dispos...
1,Q2,Les sociétés coopératives exerçant une activit...
2,Q3,Dans quels cas les dispositions de l'Acte unif...
3,Q4,Les sociétés coopératives exerçant des activit...
4,Q5,Qui peut choisir de constituer une société coo...


In [68]:
question = test_df.iloc[0]['Question']
qid = test_df.iloc[0]['ID']
# Get the answer using your existing chatbot function
a, history = create_conversation(f"{question} Ne retourne que la reponse a ma question est rien d'autre", [])
a

'Based on the provided context, the types of entities that are subject to the provisions of the Acte uniforme OHADA on the law of cooperative societies are:\n\n1. Cooperative societies: As the name suggests, cooperative societies are the entities that are subject to the provisions of the Acte uniforme OHADA. These are societies that are formed for the purpose of carrying out an activity in common, and their object is determined by their statutes.\n2. Unions of cooperative societies: Under the Acte uniforme OHADA, two or more cooperative societies with the same object can form an union of cooperative societies. This is done through the adoption of statutes by the constitutive assembly of the union, which must include at least three delegates mandated by each of the founding societies.\n3. Federations of cooperative societies: A federation of cooperative societies is an association of cooperative societies that are affiliated with each other for the purpose of pursuing common interests.\

In [67]:
_, history_ref_doc = create_conversation("Quel est le document de reference dans le quel se trouve la reponse a cette question ? Ne retourne que la reponse a ma question est rien d'autre et soit tres concis", history)
history_ref_doc[0]

("Quels types d'entités sont soumises aux dispositions de l'Acte uniforme OHADA sur le droit des sociétés coopératives ? Ne retourne que la reponse a ma question est rien d'autre",
 "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nwww.Droit-Afrique.com  OHADA \n \nActe uniforme OHADA relatif au droit des sociétés coopératives 10\nSous-section 4 - Objet de la société coopérative \nArt.20.- Toute société coopérative a un objet qui est constitué par l’activité qu’elle entreprend \net qui doit être déterminée et décrite dans ses statuts. \nL’objet de la société coopérative doit être licite. \nLorsque l’activité exercée par la société coopérative est réglementée, celle-ci doit se confor-\nmer aux règles particulières auxquelles ladite activité est soumise. \nL’objet social peut être modifié, pour chaque  forme de société coopérative, dans les condi-\ntions prévues par le pré

In [69]:
submission_sample_df = pd.read_csv("/content/SampleSubmission.csv")
submission_sample_df.head()

,ID,Target
0,Q1_Answer,NaN
1,Q1_Reference_Document,NaN
2,Q1_Article_Number,NaN
3,Q2_Answer,NaN
4,Q2_Reference_Document,NaN


In [97]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
import json

# Définition du schéma de réponse
response_schemas = [
    ResponseSchema(name="answer", description="La réponse à la question principale"),
    ResponseSchema(name="reference_document", description="Le nom du document de référence"),
    ResponseSchema(name="article_number", description="Le numéro de l'article")
]

# Création du parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Template de prompt avec formatage structuré
format_instructions = output_parser.get_format_instructions()

prompt_template = PromptTemplate(
    template="""
[INST] {question}

Réponds strictement au format suivant :
{format_instructions}

Ne fournis aucun autre texte. Aucun commentaire.
[/INST]
""",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

def get_qa_results_single_conversation(question, history=[]):
    try:
        # Générer le prompt formaté
        prompt = prompt_template.format(question=question)

        # Appel au modèle
        response, history = create_conversation(prompt, history)
        print(f"✅ Réponse du modèle : {response}")

        # Tentative de parsing structuré
        return output_parser.parse(response) | {"qid": "unknown"}

    except Exception as e:
        print(f"❌ Erreur de parsing : {e}")
        return {
            "answer": None,
            "reference_document": None,
            "article_number": None,
            "qid": "unknown",
            "error": str(e)
        }


In [ ]:
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_exponential

# 1) Wrapper avec retry pour tolérer les timeouts / erreurs LLM
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def safe_get_qa(question: str) -> dict:
    """
    Appelle get_qa_results_single_conversation avec retry.
    """
    return get_qa_results_single_conversation(question)

def process_all_questions(
    df,
    max_workers: int = 4
) -> list[dict]:
    """
    Traite en parallèle toutes les questions de `df` (colonnes 'ID' et 'Question'),
    retourne une liste de dicts avec answer, reference_document, article_number et qid.
    """
    results = []
    # Dictionnaire pour lier Future -> qid
    future_to_qid = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Soumet les tâches
        for row in df.itertuples(index=False):
            future = executor.submit(safe_get_qa, row.Question)
            future_to_qid[future] = row.ID

        # Itération avec barre de progression
        for future in tqdm(as_completed(future_to_qid),
                           total=len(future_to_qid),
                           desc="Processing Questions"):
            qid = future_to_qid[future]
            try:
                res = future.result()
                # On s'assure que le qid est bien présent
                res['qid'] = qid
            except Exception as e:
                # En cas d'échec après retry
                res = {
                    'answer': None,
                    'reference_document': None,
                    'article_number': None,
                    'qid': qid,
                    'error': str(e)
                }
            results.append(res)

    return results

# Exemple d'utilisation :
all_qa_pairs = process_all_questions(test_df, max_workers=4)


Processing Questions:   0%|          | 0/1000 [00:00<?, ?it/s]

✅ Réponse du modèle : {
"answer": "Non",
"reference_document": "Acte uniforme relatif au droit des sociétés commerciales et du groupement d'intérêt économique",
"article_number": "118"
}
✅ Réponse du modèle : {
"answer": "Non",
"reference_document": "www.droit-afrique.com",
"article_number": "118"
}

Référence document:
Acte uniforme relatif au droit des sociétés commerciales et du groupement d’intérêt économique, disponible sur le site www.droit-afrique.com.

Article number:
Article 118 de l'Acte uniforme relatif au droit des sociétés commerciales et du groupement d’intérêt économique.
✅ Réponse du modèle : {
"answer": "Toutes les personnes physiques ou morales peuvent choisir de constituer une société coopérative selon l'Acte uniforme OHADA.",
"reference_document": "www.droit-afrique.com",
"article_number": "118"
}
✅ Réponse du modèle : {
"answer": "Les entités soumises aux dispositions de l'Acte uniforme OHADA sur le droit des sociétés coopératives sont celles qui ont une structure 

In [93]:
submission_df = create_submission_file(all_qa_pairs)
submission_df.head(50)

,ID,Target
0,Q1_Answer,None
1,Q1_Reference_Document,None
2,Q1_Article_Number,None
3,Q2_Answer,None
4,Q2_Reference_Document,None
5,Q2_Article_Number,None
6,Q3_Answer,None
7,Q3_Reference_Document,None
8,Q3_Article_Number,None
9,Q4_Answer,None


# Prompt-Engineering

In [ ]:
test_query = "Qu'est-ce que l'OHADA ?"
test_prompt = f"""<s>[INST] {test_query}  [/INST] """
test_history = []
create_conversation(test_prompt, test_history)

('',
 [("<s>[INST] Qu'est-ce que l'OHADA ?  [/INST] ",
   'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nl’\'OHADA ;Attendu, selon les énonciations de l’arrêt attaqué, que suite à la saisie attribution de créances pratiquée le 30 octobre 2020 sur ses avoirs bancaires à la demande de la SCI Notre Dame de la Grâce, l’Ab A de Management dite AFRAM a fait assigner cette dernière devant le juge de l’exécution en contestation et mainlevée de la saisie ;Que le 27 novembre 2020, le juge de l’exécution a rendu son ordonnance par laquelle il a rejeté les prétentions de AFRAM et prononcé la continuation des poursuites ; que sur appel de celle-ci, la Cour d’appel judiciaire de Libreville a rendu l’arrêt objet du présent recours ;Sur le moyen unique de cassation tiré de la violation de la loiAttendu qu’il est reproché à l’arrêt attaqué d’avoir « confirmé pour le reste la décisi

In [ ]:
query = "Quel est le document de reference dans le quel se trouve la reponse a cette question ? Ne retourne que la reponse a ma question est rien d'autre"
prompt = f"""<s>[INST] {query}  [/INST] """
create_conversation(prompt, test_history)

('',
 [("<s>[INST] Qu'est-ce que l'OHADA ?  [/INST] ",
   'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nl’\'OHADA ;Attendu, selon les énonciations de l’arrêt attaqué, que suite à la saisie attribution de créances pratiquée le 30 octobre 2020 sur ses avoirs bancaires à la demande de la SCI Notre Dame de la Grâce, l’Ab A de Management dite AFRAM a fait assigner cette dernière devant le juge de l’exécution en contestation et mainlevée de la saisie ;Que le 27 novembre 2020, le juge de l’exécution a rendu son ordonnance par laquelle il a rejeté les prétentions de AFRAM et prononcé la continuation des poursuites ; que sur appel de celle-ci, la Cour d’appel judiciaire de Libreville a rendu l’arrêt objet du présent recours ;Sur le moyen unique de cassation tiré de la violation de la loiAttendu qu’il est reproché à l’arrêt attaqué d’avoir « confirmé pour le reste la décisi

# Gradio UI

In [ ]:
# Construire l'interface Gradio
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Chat with your Ohada dataset ;)')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    language = gr.State("French")
    msg.submit(create_conversation, [msg, chatbot, language], [msg, chatbot])

demo.launch()

<ipython-input-55-7fd69152d3c8>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label='Chat with your Ohada dataset ;)')


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b56b76b2e866e3fa08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Le fichier `Test.csv` contient une liste de questions auxquelles vous devez répondre à l’aide de votre LLM. Pour chaque question, vous devez indiquer quel est le Document de Référence contenant la réponse. Il faut également mentionner l’article dans lequel la réponse est donnée, si possible.

Le **Document de Référence** correspond au titre du document contenant la réponse.

Le **Numéro_Article** correspond à l'article ou aux articles dans le(s)quel(s) la réponse est mentionnée.

Vous devez soumettre vos réponses dans un fichier CSV ayant une structure similaire à celle du fichier `SampleSubmission.csv`.

In [ ]:
import pandas as pd

ss = pd.read_csv("SampleSubmission.csv")
ss.head()

,ID,Target
0,Q1_Answer,NaN
1,Q1_Reference_Document,NaN
2,Q1_Article_Number,NaN
3,Q2_Answer,NaN
4,Q2_Reference_Document,NaN


### Bonne chance pour la compétition! 🚀


### Soyez Creatifs!